In [1]:
#Importamos las librerias necesarias para crear la red neuronal
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf
#Leemos el csv
evaluation = pandas.read_csv('turkiye-student-evaluation_R_Specific.csv', header=0)

In [2]:
#Mostramos las 10 primeras lineas para ver que se ha leido bien
evaluation.head(10)

,instr,class,nb.repeat,attendance,difficulty,Q1,Q2,Q3,Q4,Q5,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28
1,1,2,1,0,4,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,1,2,1,2,4,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
5,1,2,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,1,2,1,3,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
7,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
8,1,2,1,1,3,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
9,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
10,1,2,1,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [3]:
#Seleccionamos los atributos 
atributos = evaluation.loc[:, 'class':'Q27']
atributos = atributos.to_numpy()
scaler = preprocessing.MinMaxScaler()
atributos = scaler.fit_transform(atributos)
print(atributos[5818,:])

[1.   0.   0.25 0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.  ]


In [4]:
#Seleccionamos el objetivo
objetivo = evaluation['Q28']
objetivo = evaluation['Q28'].to_numpy(dtype=np.float32)
print(objetivo)

[3. 3. 5. ... 5. 1. 1.]


In [5]:
#Separamos los datos que se usaran para entrenar y los que se usarán para evaluar los modelos
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.15)

In [6]:
normalizador = keras.layers.Normalization()#Definimos un normalizador
normalizador.adapt(atributos_entrenamiento)
red_evaluation = keras.Sequential()#Creamos la red neuronal
red_evaluation.add(keras.layers.Input(shape=(31,)))#Definimos la capa de entrada con una neurona por cada atributo
red_evaluation.add(normalizador)#Aplicamos el normalizador
red_evaluation.add(keras.layers.Dense(70, activation='relu'))#Definimos una capa con 70 neuronas y función de activación relu
red_evaluation.add(keras.layers.Dense(70, activation='sigmoid'))#Definimos una capa de 70 neuronas y función de activación sigmoide
red_evaluation.add(keras.layers.Dense(1, activation='linear')) #Defnimios una capa de salida con función de activación lienal ya que los valores esperados son números lineales entre 0 y 5

In [7]:
red_evaluation.summary()#Mostramos un resumen de la red neuronal

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 31)               63        
 n)                                                              
                                                                 
 dense (Dense)               (None, 70)                2240      
                                                                 
 dense_1 (Dense)             (None, 70)                4970      
                                                                 
 dense_2 (Dense)             (None, 1)                 71        
                                                                 
Total params: 7,344
Trainable params: 7,281
Non-trainable params: 63
_________________________________________________________________


In [8]:

red_evaluation.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])#Compilamos la red neuronal definiendo el optimizaodr, la función de perdida y dos metricas
red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)#Entrenamos la red neuronal con 10 epocas y lotes de 256 elementos

Epoch 1/10
20/20 [==============================] - 1s 4ms/step - loss: 2.0092 - accuracy: 0.1326 - mae: 1.0624
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4639 - accuracy: 0.1401 - mae: 0.5149
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.3233 - accuracy: 0.1401 - mae: 0.3784
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.2958 - accuracy: 0.1401 - mae: 0.3344
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.2872 - accuracy: 0.1401 - mae: 0.3172
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.2809 - accuracy: 0.1401 - mae: 0.3085
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.2759 - accuracy: 0.1401 - mae: 0.3047
Epoch 8/10
20/20 [==============================] - 0s 5ms/step - loss: 0.2719 - accuracy: 0.1401 - mae: 0.2993
Epoch 9/10
20/20 [==============================] - 0s 4ms/step - loss: 0.2678 - accuracy: 0.1401 - mae:

In [9]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
#Para este conjunto de datos esto practicamente no es necesario pero así nos aseguramos de cometer un error muy pequeño
perdObj = 1.0
x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
while x[0]>perdObj:
    red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)
    x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
    
print(x[0])

28/28 [==============================] - 0s 3ms/step - loss: 0.3242 - accuracy: 0.1363 - mae: 0.3103
0.324169784784317


In [10]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_evaluation.evaluate(atributos_prueba, objetivo_prueba)

28/28 [==============================] - 0s 4ms/step - loss: 0.3242 - accuracy: 0.1363 - mae: 0.3103


[0.324169784784317, 0.1363115757703781, 0.310251921415329]

In [11]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
X=atributos[5809]
predicciones = red_evaluation.predict(X)
print('Esperado 4:',predicciones)
X=atributos[5810]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5811]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5812]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5813]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5814]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5815]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5816]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos[5817]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos[5818]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5819]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)

1/1 [==============================] - 0s 239ms/step
Esperado 4: [[4.090826]]
1/1 [==============================] - 0s 59ms/step
Esperado 3: [[3.0656035]]
1/1 [==============================] - 0s 54ms/step
Esperado 3: [[3.2071948]]
1/1 [==============================] - 0s 43ms/step
Esperado 1: [[1.1087112]]
1/1 [==============================] - 0s 50ms/step
Esperado 3: [[3.0232425]]
1/1 [==============================] - 0s 53ms/step
Esperado 1: [[1.2787775]]
1/1 [==============================] - 0s 49ms/step
Esperado 1: [[1.1087112]]
1/1 [==============================] - 0s 50ms/step
Esperado 5: [[4.8623343]]
1/1 [==============================] - 0s 49ms/step
Esperado 5: [[4.9203796]]
1/1 [==============================] - 0s 50ms/step
Esperado 1: [[1.130379]]
1/1 [==============================] - 0s 57ms/step
Esperado 1: [[1.130379]]


In [12]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_evaluation = RandomForestRegressor() #Creamos el modelo
forest_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento)#Lo entrenamos
evaluaciones = forest_evaluation.predict(atributos_prueba)#Evaluamos
mse = mean_squared_error(objetivo_prueba, evaluaciones)#Calculamos la perdida
print("Error cuadrático medio (MSE) del modelo:", mse)
#Como este modelo suele tener una perdida muy pequeña tras entrenarlo una vez, no hacemos nada más para mejorarlo

Error cuadrático medio (MSE) del modelo: 0.24067985715477677


In [13]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
atributos2 = atributos[:, np.newaxis]
X=atributos2[5809]
predicciones = forest_evaluation.predict(X)
print('Esperado 4:',predicciones)
X=atributos2[5810]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5811]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5812]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5813]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5814]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5815]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5816]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos2[5817]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos2[5818]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5819]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)

Esperado 4: [4.]
Esperado 3: [3.]
Esperado 3: [3.]
Esperado 1: [1.]
Esperado 3: [3.]
Esperado 1: [1.23]
Esperado 1: [1.]
Esperado 5: [4.67166667]
Esperado 5: [5.]
Esperado 1: [1.]
Esperado 1: [1.]


In [14]:
#Importamos las librerias necesarias para crear otra red neuronal
#Volemos a importar por si se quiere ejecutar este fragmento sin ejecutar el anterior
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf

#Leemos el csv
#bikes = pandas.read_csv('hour.csv', header=0)
bikes = pandas.read_csv('hour2.csv',header = 0)


In [15]:
#Mostramos las 10 primeras lineas para ver que se ha leido bién
bikes.head(10)

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
5,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
6,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,2
7,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,1,2,3
8,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,1,7,8
9,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,14


In [16]:
#Seleccionamos los atributos y los nomalizamos
attr = bikes.loc[:, 'season':'registered']
attr = attr.to_numpy()
scaler = preprocessing.MinMaxScaler()
attr = scaler.fit_transform(attr)

print(attr)

[[0.         0.         0.         ... 0.         0.00817439 0.01467269]
 [0.         0.         0.         ... 0.         0.02179837 0.03611738]
 [0.         0.         0.         ... 0.         0.01362398 0.03047404]
 ...
 [0.         1.         1.         ... 0.19301751 0.01907357 0.09367946]
 [0.         1.         1.         ... 0.15786999 0.03542234 0.05417607]
 [0.         1.         1.         ... 0.15786999 0.03269755 0.04176072]]


In [17]:
#Seleccionamos los objetivos
obj = bikes['cnt']
obj = bikes['cnt'].to_numpy(dtype=np.float32)
print(obj)

[16. 40. 32. ... 90. 61. 49.]


In [18]:
#Cogemos un 85% de los datos para entrenar y un 15% de los datos para evaluar el modelo
(attr_entrenamiento, attr_prueba,
 obj_entrenamiento, obj_prueba) = model_selection.train_test_split(
    attr, obj, test_size=.10)

In [19]:
normalizador = keras.layers.Normalization()#Definimos un normalizador
normalizador.adapt(attr_entrenamiento)
red_bikes = keras.Sequential()#Creamos la red neuronal
red_bikes.add(keras.layers.Input(shape=(14,))) #Añadimos la capa de entrada con 14 neuronas, una por cada atributo
red_bikes.add(normalizador)#Aplicamos un normalizador
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(1, activation='linear'))#Necesitamos una capa de salida linear ya que los valores objetivos son lineales

In [20]:
#Vemos un resumen de la red
red_bikes.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 14)               29        
 ion)                                                            
                                                                 
 dense_3 (Dense)             (None, 100)               1500      
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 11,730
Trainable params: 11,701
Non-trainable params: 29
_________________________________________________________________


In [21]:
#Compilamos y entrenamos la red neuronal, buscamos minimizar la función de perdida
red_bikes.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])
red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)

Epoch 1/10
62/62 [==============================] - 1s 5ms/step - loss: 9979.7832 - accuracy: 0.0089 - mae: 65.2524
Epoch 2/10
62/62 [==============================] - 0s 4ms/step - loss: 1634.2577 - accuracy: 0.0089 - mae: 29.2513
Epoch 3/10
62/62 [==============================] - 0s 4ms/step - loss: 650.7244 - accuracy: 0.0089 - mae: 17.4556
Epoch 4/10
62/62 [==============================] - 0s 4ms/step - loss: 490.7180 - accuracy: 0.0089 - mae: 15.7866
Epoch 5/10
62/62 [==============================] - 0s 4ms/step - loss: 323.6256 - accuracy: 0.0089 - mae: 12.9286
Epoch 6/10
62/62 [==============================] - 0s 4ms/step - loss: 266.3924 - accuracy: 0.0089 - mae: 11.2974
Epoch 7/10
62/62 [==============================] - 0s 4ms/step - loss: 305.9891 - accuracy: 0.0089 - mae: 11.3284
Epoch 8/10
62/62 [==============================] - 0s 4ms/step - loss: 230.5186 - accuracy: 0.0089 - mae: 9.9287
Epoch 9/10
62/62 [==============================] - 0s 4ms/step - loss: 142.746

In [22]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
perdidaObj = 1.0
x=red_bikes.evaluate(attr_prueba, obj_prueba)
while x[0]>perdidaObj:
    red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)
    x=red_bikes.evaluate(attr_prueba, obj_prueba)
    
#La forma más efectiva que hemos encontrado para mejorar nuestra red ha sido la de hacer este bulce ya que es la unica forma de asegurarnos que la red obtendrá la precisión que busquemos
print(x[0])

55/55 [==============================] - 0s 874us/step - loss: 22.8093 - accuracy: 0.0138 - mae: 3.0984
Epoch 1/10
62/62 [==============================] - 0s 914us/step - loss: 73.0563 - accuracy: 0.0086 - mae: 6.2002
Epoch 2/10
62/62 [==============================] - 0s 3ms/step - loss: 147.9087 - accuracy: 0.0086 - mae: 8.3603
Epoch 3/10
62/62 [==============================] - 0s 1ms/step - loss: 76.6229 - accuracy: 0.0086 - mae: 5.7397 
Epoch 4/10
62/62 [==============================] - 0s 1ms/step - loss: 70.0050 - accuracy: 0.0086 - mae: 6.3591
Epoch 5/10
62/62 [==============================] - 0s 976us/step - loss: 139.8824 - accuracy: 0.0086 - mae: 7.5473
Epoch 6/10
62/62 [==============================] - 0s 917us/step - loss: 78.6147 - accuracy: 0.0086 - mae: 6.7568
Epoch 7/10
62/62 [==============================] - 0s 980us/step - loss: 55.4464 - accuracy: 0.0086 - mae: 5.2435
Epoch 8/10
62/62 [==============================] - 0s 954us/step - loss: 78.6549 - accuracy: 

62/62 [==============================] - 0s 877us/step - loss: 1.2897 - accuracy: 0.0086 - mae: 0.8621
Epoch 2/10
62/62 [==============================] - 0s 880us/step - loss: 10.3893 - accuracy: 0.0086 - mae: 2.7646
Epoch 3/10
62/62 [==============================] - 0s 936us/step - loss: 7.1189 - accuracy: 0.0086 - mae: 2.2155
Epoch 4/10
62/62 [==============================] - 0s 945us/step - loss: 2.2889 - accuracy: 0.0086 - mae: 1.0833
Epoch 5/10
62/62 [==============================] - 0s 967us/step - loss: 1.1046 - accuracy: 0.0086 - mae: 0.7967
Epoch 6/10
62/62 [==============================] - 0s 950us/step - loss: 0.6771 - accuracy: 0.0086 - mae: 0.6118
Epoch 7/10
62/62 [==============================] - 0s 927us/step - loss: 1.3230 - accuracy: 0.0086 - mae: 0.8609
Epoch 8/10
62/62 [==============================] - 0s 894us/step - loss: 7.7994 - accuracy: 0.0086 - mae: 2.3779
Epoch 9/10
62/62 [==============================] - 0s 935us/step - loss: 3.8016 - accuracy: 0.008

In [22]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_bikes.evaluate(attr_prueba, obj_prueba)

55/55 [==============================] - 0s 3ms/step - loss: 202.1550 - accuracy: 0.0109 - mae: 11.3382


[202.15499877929688, 0.010932105593383312, 11.33823299407959]

In [23]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado

X=attr[17368]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 203:',predicciones)
X=attr[17369]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 247:',predicciones)
X=attr[17370]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 315:',predicciones)
X=attr[17371]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 214:',predicciones)
X=attr[17372]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 164:',predicciones)
X=attr[17373]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 122:',predicciones)
X=attr[17374]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 119:',predicciones)
X=attr[17375]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 89:',predicciones)
X=attr[17376]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 90:',predicciones)
X=attr[17377]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 61:',predicciones)
X=attr[17378]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 49:',predicciones)

Esperado 203: [[202.57814]]
Esperado 247: [[252.82217]]
Esperado 315: [[337.69794]]
Esperado 214: [[217.0266]]
Esperado 164: [[170.1789]]
Esperado 122: [[128.54584]]
Esperado 119: [[123.768394]]
Esperado 89: [[93.56566]]
Esperado 90: [[95.27212]]
Esperado 61: [[73.397766]]
Esperado 49: [[59.368313]]


In [24]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_bikes = RandomForestRegressor()#Creamos el randomForest
forest_bikes.fit(attr_entrenamiento, obj_entrenamiento)#Lo entrenamos
evaluaciones = forest_bikes.predict(attr_prueba)#Evaluamos el modelo
mse = mean_squared_error(obj_prueba, evaluaciones)#Calculamos la perdida
print("Error cuadrático medio (MSE) del modelo:", mse)

Error cuadrático medio (MSE) del modelo: 5.394330207134642


In [25]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
attr2 = attr[:, np.newaxis]
X=attr2[17368]
predicciones = forest_bikes.predict(X)
print('Esperado 203:',predicciones)
X=attr2[17369]
predicciones = forest_bikes.predict(X)
print('Esperado 247:',predicciones)
X=attr2[17370]
predicciones = forest_bikes.predict(X)
print('Esperado 315:',predicciones)
X=attr2[17371]
predicciones = forest_bikes.predict(X)
print('Esperado 214:',predicciones)
X=attr2[17372]
predicciones = forest_bikes.predict(X)
print('Esperado 164:',predicciones)
X=attr2[17373]
predicciones = forest_bikes.predict(X)
print('Esperado 122:',predicciones)
X=attr2[17374]
predicciones = forest_bikes.predict(X)
print('Esperado 119:',predicciones)
X=attr2[17375]
predicciones = forest_bikes.predict(X)
print('Esperado 89:',predicciones)
X=attr2[17376]
predicciones = forest_bikes.predict(X)
print('Esperado 90:',predicciones)
X=attr2[17377]
predicciones = forest_bikes.predict(X)
print('Esperado 61:',predicciones)
X=attr2[17378]
predicciones = forest_bikes.predict(X)
print('Esperado 49:',predicciones)

Esperado 203: [202.55]
Esperado 247: [247.13]
Esperado 315: [314.04]
Esperado 214: [214.01]
Esperado 164: [163.97]
Esperado 122: [122.27]
Esperado 119: [119.14]
Esperado 89: [88.88]
Esperado 90: [89.96]
Esperado 61: [61.14]
Esperado 49: [48.88]


In [26]:
#Metodo LIME
import random
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPClassifier, MLPRegressor



"""
Partiremos de este seudocódigo para implementar el metodo
N es el n´umero de permutaciones a realizar
f es el modelo a explicar
X′ ← {} muestras perturbadas
R ← {} representaciones
W ← {} las distancias entre la muestra x y sus perturbaciones
for 1
to
N do
Selecciona k atributos aleatoriamente
x′ ← una perturbaci´on de x donde se perturban los k atributos anteriores.
w ← la distancia entre x y x′
r ← la representaci´on de x′ respecto a x
X′ ← X′ ∪ x′
R ← R ∪ r
W ← R ∪ w
end for
Y ′ ← f(X′) las predicciones de las perturbaciones
G ← modelo ridge entrenado con R para predecir Y ′ y ponderando cada
muestra con W
return los par´ametros de G

"""

def explain_model(f, x, N,M):
    #f es el modelo a explicar
    #X es una lista de ejemplos a los que se aplicará LIME
    #N es el número de permutaciones a realizar
    #M contiene todos los ejemplos
    #Si queremos que el algoritmo cumpla la metrica identidad necesitamos fijar semillas para la aleatoriedad
    random.seed(11)
    Xi = [] #Aqui guardaremos las muestras perturbadas
    R = []  #Aqui guardaremos las representaciones
    W = []  #Aqui guradaremos las distancias
    for i in range(N):
        k = random.randint(1, len(x)) #Escojo un número aletario que representa el número de los atributos a seleccionar
        perturbed_x = x.copy() #copio la muestra original
        for j in range(k): 
            perturbed_attr = random.randint(0,len(x)-1)#Voy escojiento los atributos que perturbare de forma aleatoria
            mx = abs(max([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor máximo
            mn = abs(min([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor mínimo
            perturbed_x[perturbed_attr] = random.uniform(mn, mx) # pertubo el atributo con un valor aleatorio ente mn y mx para que el valor esté acotado
        w = abs(sum([x[attr] - perturbed_x[attr] for attr in range(0,len(x)-1)])) #Calculo la distancia entre el original y el perturbado
        r = [0  if perturbed_x[attr] == x[attr] else 1 for attr in range(0,len(x)-1)] #Calculo la representación del perturbado respecto al original utilizando un operador ternario
        Xi.append(perturbed_x) #Acumulo en la lista los perturbados
        R.append(r)#Acumulamos en la lista de representaciones
        W.append(w)#Añadimos las distancias a la lista
    Y_perturbed = []#Aqui guardaremos las predicciones de las perturbaciones
    for i in range(len(Xi)):
        y = 1
            #En los siguiente if y else comprobamos el tipo de modelo que tenemos ya que dependiendo del tipo necesita recibir los atributos de una forma u otra
        if isinstance(f, keras.Sequential):#Comprueba si el modelo es una red neuronal
            xi = Xi[i]
            y = f.predict(xi,verbose=0)#Hacemos la prediccion con verbose=0 para no cargar la salida de lineas generadas por keras
        else: #Si no, es un randomForest
            xi = Xi[i]
            array = np.array(xi)
            xi = array.reshape(1, -1)
            y = f.predict(xi)#RandomForest no tiene verbose porque no genera nada
                
            
        Y_perturbed.append(y) #Aplico el modelo f a los ejemplos perturbados
            
    Y_perturbed = np.squeeze(Y_perturbed) #Por alguna razón aparece el error Found array with dim 3. Estimator expected <= 2" y lo arreglamos con esta linea que quita una dimensión al array
    G = Ridge()
    G.fit(R, Y_perturbed, sample_weight=W)
    return G.coef_#Devolvemos los parametros


In [27]:
#Hacemos varias pruebas para comprobar que funciona el metodo LIME
atributivos = atributos[5809]

explain_model(red_evaluation,atributivos,10,atributos)

array([-0.10184417,  0.00241792,  0.04822971, -0.02986165, -0.01276533,
       -0.01580204, -0.0762422 , -0.01357334, -0.00090499, -0.0250274 ,
        0.02200894,  0.01847016,  0.01755265, -0.00693519, -0.05215315,
       -0.01139149, -0.00273589, -0.05908835, -0.14535998,  0.04609799,
       -0.03014421,  0.0041993 , -0.04061819,  0.13808697, -0.0762422 ,
       -0.06911779,  0.02317153,  0.00266812, -0.03989109,  0.04675851])

In [28]:
#Hacemos varias pruebas para comprobar que funciona el metodo LIME
atributivos = atributos[5809]

explain_model(red_evaluation,atributivos,10,atributos)

array([-0.10184417,  0.00241792,  0.04822971, -0.02986165, -0.01276533,
       -0.01580204, -0.0762422 , -0.01357334, -0.00090499, -0.0250274 ,
        0.02200894,  0.01847016,  0.01755265, -0.00693519, -0.05215315,
       -0.01139149, -0.00273589, -0.05908835, -0.14535998,  0.04609799,
       -0.03014421,  0.0041993 , -0.04061819,  0.13808697, -0.0762422 ,
       -0.06911779,  0.02317153,  0.00266812, -0.03989109,  0.04675851])

In [29]:
atributivos = atributos[5809]

explain_model(forest_evaluation,atributivos,10,atributos)

array([-0.04567316,  0.03737029, -0.01747319, -0.00581188,  0.04781538,
        0.02042588, -0.05512401, -0.02585784, -0.03849429, -0.04438821,
        0.00195844, -0.01971897, -0.05228054,  0.02870812, -0.06345086,
       -0.02553086,  0.01325077, -0.03474274, -0.05530213, -0.00636841,
       -0.06149242, -0.01545736, -0.05446171,  0.03635467, -0.05512401,
       -0.00017812, -0.04088842,  0.00070691,  0.02875273, -0.04350318])

In [30]:
attri = attr[17378]

explain_model(red_bikes,attri,10,attr)

array([ -19.47316344,  -87.62582367,  173.46235445,   72.00970661,
         25.99622145, -146.04761705,  112.50414511,  -88.94645408,
         42.05197305,   10.47798993,  -49.95925407,  -36.57972304,
        136.64476639])

In [31]:
attri = attr[17378]

explain_model(forest_bikes,attri,10,attr)

array([ -26.43536962,  -96.3630076 ,  190.24351781,   62.92365087,
         50.78609023, -161.15452103,  148.2345355 , -112.14155161,
         19.56360321,  -20.12890397,  -51.61876086,  -65.18220811,
        127.1836651 ])

In [32]:
#En la definición de cada metrica se importarán las cosas necesarias por si solo se quiere probar una metrica
import numpy as np


def identidad(x1,x2,exp1,exp2):
    #Definición: El principio de identidad establece que objetos idénticos deben recibir explicaciones idénticas.

    res = True #Si los objetos no son identicos no hay que valuar
    distance = np.linalg.norm(x1 - x2)#Calcula las distancias entre los dos ejemplos
    if(distance ==0):   #Solo comprobamos si las explicaciones sean identicas si los objetos son identicos
        res = np.linalg.norm(exp1 - exp2) ==0#Comprobamos si ambas explicaciones son iguales
    return res

In [33]:
ejemplo2=attr[17375]
ejemplo1 = attr[17375]
print(ejemplo2)
print(ejemplo1)
exp1 = explain_model(forest_bikes,ejemplo1,2,attr)
exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
print(identidad(ejemplo1,ejemplo2,exp1,exp2))

[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
True


In [34]:
import numpy as np


def separabilidad(x1,x2,exp1,exp2):
#Definición:#Separabilidad: Objetos no idénticos no pueden tener explicaciones idénticas. Para simplificar, cada característica 
#tiene un nivel mínimo de importancia, positivo o negativo, en las predicciones.

    res = True
    distance = np.linalg.norm(x1 - x2)#Calcula las distancias entre los dos ejemplos
    if(distance >0):   #Si la distancia es distinta de cero
        res = not(np.linalg.norm(exp1 - exp2) ==0)#Si las explicaciones son distintas se cumple la separabilidad
    
    return res

In [35]:
ejemplo2=attr[17376]
ejemplo1 = attr[17375]
print(ejemplo2)
print(ejemplo1)
exp1 = explain_model(forest_bikes,ejemplo1,2,attr)
exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
print(separabilidad(ejemplo1,ejemplo2,exp1,exp2))

[0.         1.         1.         0.91304348 0.         0.16666667
 1.         0.         0.24489796 0.2576     0.6        0.19301751
 0.01907357 0.09367946]
[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
True


In [36]:
import numpy as np
from scipy.stats import spearmanr

def estabilidad(x1,x2,exp1,exp2,umbEj,umbExp):
    #Objetos similares deben tener explicaciones similares
    #Para esto calcularemos la correlacion de Spearman para los ejemplos, si superan el limite que se pasará a la función
    #Calcularemos la correlación de Spearman para las explicaciones y comprobaremos si supera el umbral, en ese caso, cumpliran la estabilidad
    res = True
    correlacionEj, _ = spearmanr(x1, x2)
    if(abs(correlacionEj) >= umbEj):
        correlacionExp, _ = spearmanr(exp1,exp2)
        res = abs(correlacionExp)>=umbExp 
    return res

In [37]:
ejemplo2=attr[17376]
ejemplo1 = attr[17375]
print(ejemplo2)
print(ejemplo1)
print(spearmanr(ejemplo1, ejemplo2))
exp1 = explain_model(forest_bikes,ejemplo1,2,attr)
exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
print(spearmanr(exp1, exp2))
print(estabilidad(ejemplo1,ejemplo2,exp1,exp2,0.75,0.75))

[0.         1.         1.         0.91304348 0.         0.16666667
 1.         0.         0.24489796 0.2576     0.6        0.19301751
 0.01907357 0.09367946]
[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
SpearmanrResult(correlation=0.8770812523243259, pvalue=3.801615077865054e-05)
SpearmanrResult(correlation=1.0, pvalue=0.0)
True


In [84]:
import numpy as np
from scipy import integrate

def selectividad(X, model):
    X = X.reshape(1, -1)
    # Obtener las predicciones originales
    y_pred_original = model.predict(X)
    
    # Calcular el AUC original
    auc_original = integrate.trapz(y_pred_original)
    
    # Obtener la importancia de las características del modelo
    feature_importance = model.feature_importances_
    
    # Ordenar las características de más a menos relevante
    feature_order = np.argsort(np.abs(feature_importance))[::-1]
    
    # Inicializar una lista para almacenar los AUCs
    auc_list = [auc_original]
    
    # Iterar sobre las características y calcular los AUCs sin cada una de ellas
    for i in range(1, len(feature_order) + 1):
        # Crear una copia de los datos de entrada X y establecer la característica en cero
        X_modified = X.copy()
        X_modified[:, feature_order[:i]] = 0
        
        # Obtener las predicciones modificadas
        y_pred_modified = model.predict(X_modified)
        
        # Calcular el AUC modificado
        auc_modified = integrate.trapz(y_pred_modified)
        
        # Almacenar el AUC modificado en la lista
        auc_list.append(auc_modified)
    
    # Calcular el AUC promedio de todas las iteraciones
    selectivity = np.mean(auc_original - auc_list)
    
    return selectivity


In [88]:
X = atributos[5809,:]
selectividad(X,forest_evaluation)

0.0

In [38]:
def coherencia(mae1, mae2):
    #Se calcula la diferencia entre el error de predicción (mae1) sobre la señal original y el error de predicción mae2
    #de una nueva señal donde se eliminan las características no importantes.
    alpha_i = abs(mae1 - mae2)
    return alpha_i


In [39]:
def completitud(ei, pi):
    #Evalúa el porcentaje de error de explicación con respecto al error de predicción.
    gamma_i = ei / pi
    return gamma_i

In [40]:
import numpy as np

def congruencia(coherencias):
    #Esta métrica ayuda a capturar la variabilidad de la coherencia.
    coherencia_media = np.mean(coherencias)
    res = (sum((c - coherencia_media)**2 for c in coherencias)/len(coherencias))*0.5
    return res

In [41]:
#Comprobaremos la métrica identidad
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y para cada ejemplo en cada modelo generaremos la explicacion dos veces y comprobaremos que es la misma explicación
atributivos = atributos[5553:5809,:]
sonIguales = True
for x in atributivos:
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    exp2 = explain_model(forest_evaluation,x,2,atributos)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break
if(sonIguales):
    print("El modelo RandomForest de evaluación cumple la métrica identidad")
else:
    print("El modelo RandomForest de evaluación no cumple la métrica identidad")

sonIguales=True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,x,2,atributos)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break

if(sonIguales):
    print("El modelo Red neuronal de evaluación cumple la métrica identidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la métrica identidad")

X=attr[17122:17378,:]
sonIguales=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,x,2,attr)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break
    
if(sonIguales):
    print("El modelo Red neuronal de bikes cumple la métrica identidad")
else:
    print("El modelo Red neuronal de bikes no cumple la métrica identidad")
    
sonIguales=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,x,2,attr)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break
    
if(sonIguales):
    print("El modelo RandomForest de bikes cumple la métrica identidad")
else:
    print("El modelo RandomForest de bikes no cumple la métrica identidad")

El modelo RandomForest de evaluación cumple la métrica identidad
El modelo Red neuronal de evaluación cumple la métrica identidad
El modelo Red neuronal de bikes cumple la métrica identidad
El modelo RandomForest de bikes cumple la métrica identidad


In [ ]:
#Comprobaremos la métrica separabilidad
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y los comparamos con otro ejemplo
ejemplo1=atributos[5552]
atributivos = atributos[5553:5809,:]
cumpleSeparabilidad = True
for x in atributivos:
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    exp2 = explain_model(forest_evaluation,ejemplo1,2,atributos)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo1,exp1,exp2)
    if(not cumpleSeparabilidad):
        break
if(cumpleSeparabilidad):
    print("El modelo RandomForest de evaluación cumple la métrica separabilidad")
else:
    print("El modelo RandomForest de evaluación no cumple la métrica separabilidad")

cumpleSeparabilidad = True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,ejemplo1,2,atributos)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo1,exp1,exp2)
    if(not cumpleSeparabilidad):
        break

if(cumpleSeparabilidad):
    print("El modelo Red neuronal de evaluación cumple la métrica separabilidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la métrica separabilidad")

ejemplo2 = attr[17121]
X=attr[17122:17378,:]
cumpleSeparabilidad=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,ejemplo2,2,attr)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo2,exp1,exp2)
    if(not cumpleSeparabilidad):
        break
    
if(cumpleSeparabilidad):
    print("El modelo Red neuronal de bikes cumple la métrica separabilidad")
else:
    print("El modelo Red neuronal de bikes no cumple la métrica separabilidad")
    
cumpleSeparabilidad=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo2,exp1,exp2)
    if(not cumpleSeparabilidad):
        break
    
if(cumpleSeparabilidad):
    print("El modelo RandomForest de bikes cumple la métrica separabilidad")
else:
    print("El modelo RandomForest de bikes no cumple la métrica separabilidad")

In [ ]:
#Comprobaremos la métrica estabilidad
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y los comparamos con otro ejemplo, del cual sabemos que hay ejemplos muy similares

ejemplo1=atributos[5819]
atributivos = atributos[5553:5819,:]
cumpleEstabilidad = True
for x in atributivos:
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    exp2 = explain_model(forest_evaluation,ejemplo1,2,atributos)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo1,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break
if(cumpleEstabilidad):
    print("El modelo RandomForest de evaluación cumple la métrica separabilidad")
else:
    print("El modelo RandomForest de evaluación no cumple la métrica separabilidad")

cumpleEstabilidad = True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,ejemplo1,2,atributos)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo1,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break

if(cumpleEstabilidad):
    print("El modelo Red neuronal de evaluación cumple la métrica separabilidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la métrica separabilidad")

ejemplo2 = attr[17376]
X=attr[17122:17378,:]
cumpleEstabilidad=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,ejemplo2,2,attr)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo2,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break
    
if(cumpleEstabilidad):
    print("El modelo Red neuronal de bikes cumple la métrica estabilidad")
else:
    print("El modelo Red neuronal de bikes no cumple la métrica estabilidad")
    
cumpleEstabilidad=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo2,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break
    
if(cumpleEstabilidad):
    print("El modelo RandomForest de bikes cumple la métrica estabilidad")
else:
    print("El modelo RandomForest de bikes no cumple la métrica estabilidad")

In [44]:
#Para probar la coherencia lo que haremos será para varias muestras muestras, generar una predicción, generar su explicación 
#y a partir de su explicación quitaremos los N atributos menos importantes y generaremos otra explicación, a partir de ambas predicciones 
#calcularemos dos MAE y con ello calcularemos la coherencia
from sklearn.metrics import mean_absolute_error


atributivos = atributos[5810:5819,:]
objetivos = objetivo[5810:5819]

N=2
#predicciones1 = []
#predicciones2 = []
coherencias = []

for i in range(len(atributivos)):
    x = atributivos[i]
    y = x.copy()
    exp1 = explain_model(red_evaluation,x,2,atributos)
    prediccion1 = red_evaluation.predict(x,verbose=0)
    #predicciones1.append(prediccion1)np.squeeze(predicciones1)  
    errp = abs(objetivos[i]-prediccion1)
   
    for j in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 99999999999999999
    prediccion2 = red_evaluation.predict(y,verbose=0)
   
    errexp = abs(objetivos[i]-prediccion2)
    coherencias.append(abs(errp-errexp))



    

print("El modelo Red Neuronal de evaluación tiene unas coherencias de ",coherencias)

N=2
coherencias = []

for i in range(len(atributivos)):
    x = atributivos[i]
    y = x.copy()
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    x = x.reshape(1, -1)
    prediccion1 = forest_evaluation.predict(x)
    errp = abs(objetivos[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 999999999999999999
    y = y.reshape(1, -1)
    prediccion2 = forest_evaluation.predict(y)
      
    errexp = abs(objetivos[i]-prediccion2)
    coherencias.append(abs(errp-errexp))
    

print("El modelo RandomForest de evaluación tiene una coherencia de ",coherencias)

X=attr[17360:17378,:]
objs = obj[17360:17378]


N=2
coherencias = []


for i in range(len(X)):
    x = X[i]
    y = x.copy()
    exp1 = explain_model(red_bikes,x,2,attr)
    prediccion1 = red_bikes.predict(x,verbose=0)
    errp = abs(objs[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 99999999999999999
    prediccion2 = red_bikes.predict(y,verbose=0)
       
    errexp = abs(objs[i]-prediccion2)
    coherencias.append(abs(errp-errexp))


    

print("El modelo Red Neuronal de bikes tiene unas coherencias de ",coherencias)


N=2
coherencias = []

for i in range(0,len(X)):
    x = X[i]
    y = x.copy()
    exp1 = explain_model(forest_bikes,x,2,attr)
    x = x.reshape(1, -1)
    prediccion1 = forest_bikes.predict(x)
    errp = abs(objs[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 999999999999999999
    y = y.reshape(1, -1)
    prediccion2 = forest_bikes.predict(y)
    errexp = abs(objs[i]-prediccion2)
    coherencias.append(abs(errp-errexp))
    

    

print("El modelo RandomForest de bikes tiene una coherencia de ",coherencias)


El modelo Red Neuronal de evaluación tiene unas coherencias de  [array([[0.5243423]], dtype=float32), array([[0.382751]], dtype=float32), array([[0.03398597]], dtype=float32), array([[0.5667033]], dtype=float32), array([[0.04376876]], dtype=float32), array([[0.03398597]], dtype=float32), array([[3.3750787]], dtype=float32), array([[3.396226]], dtype=float32), array([[0.01231825]], dtype=float32)]
El modelo RandomForest de evaluación tiene una coherencia de  [array([0.16]), array([0.16]), array([2.]), array([0.16]), array([1.42]), array([2.]), array([1.19166667]), array([1.52]), array([2.])]
El modelo Red Neuronal de bikes tiene unas coherencias de  [array([[4.9194164]], dtype=float32), array([[22.228912]], dtype=float32), array([[62.01821]], dtype=float32), array([[151.0502]], dtype=float32), array([[119.67427]], dtype=float32), array([[70.92533]], dtype=float32), array([[93.513885]], dtype=float32), array([[143.5669]], dtype=float32), array([[143.00076]], dtype=float32), array([[148.2

In [43]:
from sklearn.metrics import mean_absolute_error


atributivos = atributos[5810:5819,:]
objetivos = objetivo[5810:5819]

atributos2 = evaluation.loc[:, 'class':'Q27']
atributos2 = atributos2.to_numpy()
atributos2 = atributos2[5810:5819,:]
predicciones1 = []
predicciones2 = []
for j in range(len(atributivos)):
    x = atributivos[j]
    pred1 = red_evaluation.predict(x,verbose = 0)
    predicciones1.append(pred1)
    y = 0
    exp1 = explain_model(red_evaluation,x,2,atributos)
    atributo2 = atributos2[j]
    for i in range(len(exp1)):
        y += exp1[i]*atributo2[i]
    predicciones2.append(y)

predicciones1 = np.squeeze(predicciones1)  
predicciones2 = np.squeeze(predicciones2) 
mae1 = mean_absolute_error(objetivos, predicciones1)
mae2 = mean_absolute_error(objetivos, predicciones2)

completitud = mae2/mae1
print("El modelo Red Neuronal de evaluación tiene unas completitudes de ",completitud)

predicciones1 = []
predicciones2 = []
for j in range(len(atributivos)):
    x = atributivos[j]
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    x = x.reshape(1,-1)
    pred1 = forest_evaluation.predict(x)
    predicciones1.append(pred1)
    y = 0
    atributo2 = atributos2[j]
    for i in range(len(exp1)):
        y += exp1[i]*atributo2[i]
    predicciones2.append(y)

predicciones1 = np.squeeze(predicciones1)  
predicciones2 = np.squeeze(predicciones2) 
mae1 = mean_absolute_error(objetivos, predicciones1)
mae2 = mean_absolute_error(objetivos, predicciones2)

completitud = mae2/mae1
print("El modelo RandomForest de evaluación tiene una completitud de ",completitud)


X = attr[17360:17378,:]
objs = obj[17360:17378]

attr2 = bikes.loc[:, 'season':'registered']
attr2 = attr2.to_numpy()
attr2 = attr2[17360:17378,:]
predicciones1 = []
predicciones2 = []
for j in range(len(X)):
    x = X[j]
    pred1 = red_bikes.predict(x,verbose = 0)
    predicciones1.append(pred1)
    y = 0
    exp1 = explain_model(red_bikes,x,2,attr)
    atributo2 = attr2[j]
    for i in range(len(exp1)):
        y += exp1[i]*atributo2[i]
    predicciones2.append(y)

predicciones1 = np.squeeze(predicciones1)  
predicciones2 = np.squeeze(predicciones2) 
mae1 = mean_absolute_error(objs, predicciones1)
mae2 = mean_absolute_error(objs, predicciones2)

completitud = mae2/mae1
print("El modelo Red Neuronal de bikes tiene una completitud de ",completitud)

predicciones1 = []
predicciones2 = []
for j in range(len(X)):
    x = X[j]
    exp1 = explain_model(forest_bikes,x,2,atributos)
    x = x.reshape(1,-1)
    pred1 = forest_bikes.predict(x)
    predicciones1.append(pred1)
    y = 0
    atributo2 = attr2[j]
    for i in range(len(exp1)):
        y += exp1[i]*atributo2[i]
    predicciones2.append(y)

predicciones1 = np.squeeze(predicciones1)  
predicciones2 = np.squeeze(predicciones2) 
mae1 = mean_absolute_error(objs, predicciones1)
mae2 = mean_absolute_error(objs, predicciones2)

completitud = mae2/mae1
print("El modelo RandomForest de bikes tiene una completitud de ",completitud)
    

El modelo Red Neuronal de evaluación tiene unas completitudes de  18.857535116967213
El modelo RandomForest de evaluación tiene una completitud de  42.918355104134285
El modelo Red Neuronal de bikes tiene una completitud de  28.454655691093162
El modelo RandomForest de bikes tiene una completitud de  824.8968335627013


In [42]:

from sklearn.metrics import mean_absolute_error
import numpy as np


atributivos = atributos[5810:5819,:]
objetivos = objetivo[5810:5819]

N=2
#predicciones1 = []
#predicciones2 = []
coherencias = []

for i in range(len(atributivos)):
    x = atributivos[i]
    y = x.copy()
    exp1 = explain_model(red_evaluation,x,2,atributos)
    prediccion1 = red_evaluation.predict(x,verbose=0)
    #predicciones1.append(prediccion1)np.squeeze(predicciones1)  
    errp = abs(objetivos[i]-prediccion1)
   
    for j in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 99999999999999999
    prediccion2 = red_evaluation.predict(y,verbose=0)
   
    errexp = abs(objetivos[i]-prediccion2)
    coherencias.append(abs(errp-errexp))
    

coherencias = list(coherencias)
congr = congruencia(coherencias)

    

print("El modelo Red Neuronal de evaluación tiene una congruencia de ",congr)

N=2
coherencias = []

for i in range(len(atributivos)):
    x = atributivos[i]
    y = x.copy()
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    x = x.reshape(1, -1)
    prediccion1 = forest_evaluation.predict(x)
    errp = abs(objetivos[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 999999999999999999
    y = y.reshape(1, -1)
    prediccion2 = forest_evaluation.predict(y)
      
    errexp = abs(objetivos[i]-prediccion2)
    coherencias.append(abs(errp-errexp))
    
coherencias = list(coherencias)
congr = congruencia(coherencias)

print("El modelo RandomForest de evaluación tiene una congruencia de ",congr)

X=attr[17360:17378,:]
objs = obj[17360:17378]


N=2
coherencias = []


for i in range(len(X)):
    x = X[i]
    y = x.copy()
    exp1 = explain_model(red_bikes,x,2,attr)
    prediccion1 = red_bikes.predict(x,verbose=0)
    errp = abs(objs[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 99999999999999999
    prediccion2 = red_bikes.predict(y,verbose=0)
       
    errexp = abs(objs[i]-prediccion2)
    coherencias.append(abs(errp-errexp))

coherencias = list(coherencias)
congr = congruencia(coherencias)    

print("El modelo Red Neuronal de bikes tiene una congruencia de ",congr)


N=2
coherencias = []

for i in range(0,len(X)):
    x = X[i]
    y = x.copy()
    exp1 = explain_model(forest_bikes,x,2,attr)
    x = x.reshape(1, -1)
    prediccion1 = forest_bikes.predict(x)
    errp = abs(objs[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 999999999999999999
    y = y.reshape(1, -1)
    prediccion2 = forest_bikes.predict(y)
    errexp = abs(objs[i]-prediccion2)
    coherencias.append(abs(errp-errexp))
    
coherencias = list(coherencias)
congr = congruencia(coherencias)

print("El modelo RandomForest de bikes tiene una congruencia de ",congr)


El modelo Red Neuronal de evaluación tiene una congruencia de  [[0.88277394]]
El modelo RandomForest de evaluación tiene una congruencia de  [0.29509602]
El modelo Red Neuronal de bikes tiene una congruencia de  [[1289.5055]]
El modelo RandomForest de bikes tiene una congruencia de  [1547.76981111]
